# RAG and Semantic Retrieval on a Single Document

Deep Search allows users to interact with the documents using conversational AI, i.e. you interact with a virtual assistant which answer your questions using the information in the document.

In this example we demonstrate how achive the same interaction programmatically.

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.


### GenAI Integration required

When interacting with the virtual assistant, Deep Search requires a connection to a Generative AI API. Currently, we support connections to [watsonx.ai](https://www.ibm.com/products/watsonx-ai) or the IBM-internal GenAI platform BAM.

Deep Search allows custom GenAI configurations for each project.
In the following example you will require to work in a project which has such GenAI capabilities activated.

### Set notebook parameters


In [1]:
from dsnotebooks.settings import DocQANotebookSettings

# notebooks settings auto-loaded from .env / env vars
notebook_settings = DocQANotebookSettings()

PROFILE_NAME = notebook_settings.profile     # the profile to use
PROJ_KEY = notebook_settings.proj_key        # the project to use

# index and doc for doc QA from semantically indexed collection
SEM_ON_IDX_KEY = notebook_settings.sem_on_idx_key
SEM_ON_IDX_DOC_HASH = notebook_settings.sem_on_idx_doc_hash

# index and doc for doc QA from not semantically indexed collection
SEM_OFF_IDX_KEY = notebook_settings.sem_off_idx_key
SEM_OFF_IDX_DOC_HASH = notebook_settings.sem_off_idx_doc_hash

SKIP_INGESTED_DOCS = notebook_settings.skip_ingested_docs  # whether to skip any already semantically ingested docs

RETR_K = notebook_settings.retr_k            # the number of search results to retrieve
TEXT_WEIGHT = notebook_settings.text_weight  # the weight of lexical search (0.0: semantic-only, 1.0: lexical-only, anything in between: hybrid search)
RERANK = notebook_settings.rerank            # whether to rerank the search results
RAISE = notebook_settings.raise_on_sem_err   # whether semantic operation errors should raise an exception or be reflected in response fields

### Import example dependencies

In [2]:
# Import standard dependenices
import rich

# IPython utilities
from IPython.display import display, Markdown

# Import the deepsearch-toolkit
from deepsearch.cps.client.api import CpsApi
from deepsearch.cps.client.components.elastic import ElasticProjectDataCollectionSource
from deepsearch.cps.queries import RAGQuery, SemanticQuery
from deepsearch.cps.queries.results import RAGResult, SearchResult, SearchResultItem


### Connect to Deep Search

In [3]:
api = CpsApi.from_env(profile_name=PROFILE_NAME)


### Utils

In [4]:
def render_provenance_url(
    api: CpsApi,
    coords: ElasticProjectDataCollectionSource,
    retr_item: SearchResultItem,
):
    ## compute URL to the document in the Deep Search UI
    item_index = int(retr_item.main_path[retr_item.main_path.rfind(".")+1:])
    doc_url = api.documents.generate_url(
        document_hash=retr_item.doc_hash,
        data_source=coords,
        item_index=item_index,
    )
    display(Markdown(f"The provenance of the answer can be inspected on the [source document]({doc_url})."))

---

## QA on document within semantically indexed collection

### Prepare data source

In [5]:
from deepsearch.cps.client.components.documents import PrivateDataDocumentSource

coords = ElasticProjectDataCollectionSource(
    proj_key=PROJ_KEY,
    index_key=SEM_ON_IDX_KEY,
)
data_source = PrivateDataDocumentSource(
    source=coords,
    document_hash=SEM_ON_IDX_DOC_HASH,
)

### RAG

If the document is part of a semantically indexed collection (see [Document Collection QA](https://github.com/DS4SD/deepsearch-examples/tree/main/examples/qa_doc_collection) for details),
we can directly do RAG on it as shown below:

In [6]:
question = "Where was the first European IBM research lab located?"

# submit natural-language query on document
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=RETR_K,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='The first European IBM research lab was located in Adliswil, Switzerland, near Zurich.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, 
ZRL) is the European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, 
Switzerland.',
                        main_path='main-text.70',
                        path_group=['main-text.69', 'main-text.70'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.58',
                    'main-text.59',
                    'main-text.60',
                    'main-text.61',
                    'main-text.62',
                    'main-text.63',
                    'main-text.64',
                    'main-text.65',
                    'main-text.66',
                    'main-text.67',
                    'main-text.68',
                    'main-text.69',
                    'main-text.70',
                    'main-text.71',
                    'main-text.72',
                    'main-text.73',
                    'main-text.74',
                    'main-text.75',
                    'main-text.76'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk="Israel\nIBM Research-Haifa, previously known as the Haifa Research Lab (HRL) was founded as a 
small scientific center in 1972. $^{[55]}$ Since then, it has grown into a major lab that leads the development of 
innovative technologies and solutions for the IBM corporation. The lab's offices are situated in three locations 
across Israel: Haifa, Tel Aviv, and Beer Sheva.",
            main_path='main-text.62',
            path_group=['main-text.61', 'main-text.62'],
            source_is_text=True
        )
    ]
)

Additionally, we can generate a provenance URL to the document in the Deep Search UI:

In [7]:
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.retr_items[0])

The provenance of the answer can be inspected on the [source document](https://cps.foc-deepsearch.zurich.ibm.com/projects/e0ea87922f4b732407fb3b9cf3475f0edb90cc2d/library/private/d70f151acff22f19f9cfaffb1f5baa810c8de3db?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjJkNzBmMTUxYWNmZjIyZjE5ZjljZmFmZmIxZjViYWE4MTBjOGRlM2RiJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyYjMwYmM2NjdhMzI0YWUxMTFkMDI1NTI2NTYzYjY3NGE4ZDNmZDg2OWJjMDdjOGZkMjA0YWE5NWIwNWQ0MWYwYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMmIzMGJjNjY3YTMyNGFlMTExZDAyNTUyNjU2M2I2NzRhOGQzZmQ4NjliYzA3YzhmZDIwNGFhOTViMDVkNDFmMGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0E3MCU3RCU3RA%3D%3D).

Let us try out a different question on our document corpus.
Here we also include (commented out) various additional parameters the user can optionally set:
- `retr_k`: number of items to retrieve
- `text_weight`: weight of lexical search (`0.0`: fully semantic search, `1.0`: fully lexical search, anything in-between: hybrid search)
- `rerank`: whether to rerank the retrieval results
- `gen_ctx_extr_method` (Literal["window", "page"], optional): method for gen context extraction from document; defaults to "window"
- `gen_ctx_window_size` (int, optional): (relevant only if `gen_ctx_extr_method` is "window") max chars to use for extracted gen context (actual extraction quantized on doc item level); defaults to 5000
- `gen_ctx_window_lead_weight` (float, optional): (relevant only if `gen_ctx_extr_method` is "window") weight of leading text for distributing remaining window size after extracting the `main_path`; defaults to 0.5 (centered around `main_path`)
- `return_prompt` (bool, optional): whether to return the instantiated prompt; defaults to False

For more details refer to `deepsearch.cps.queries.RAGQuery`.

In [8]:
question = "How many research labs does IBM have?"

# submit natural-language query on document
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    rerank=RERANK,

    ## optional generation params
    # model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q",
    # gen_params={"random_seed": 42, "max_new_tokens": 1024},
    # prompt_template="Answer the query based on the context.\n\nContext: {{ context }}\n\nQuery: {{ query }}",

    # gen_ctx_extr_method="window",
    # gen_ctx_window_size=5000,
    # gen_ctx_window_lead_weight=0.5
    # return_prompt=True,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='IBM Research holds 12 research labs across 6 continents.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        chunk='(Redirected from IBM Almaden Research Center)\nIBM Research is the research and 
development division for IBM, an American multinational information technology company headquartered in Armonk, New
York, with operations in over 170 countries. IBM Research is the largest industrial research organization in the 
world and has twelve labs on six continents. [1]',
                        main_path='main-text.3',
                        path_group=['main-text.2', 'main-text.3'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.1',
                    'main-text.2',
                    'main-text.3',
                    'main-text.4',
                    'main-text.5',
                    'main-text.7',
                    'main-text.8',
                    'main-text.10',
                    'main-text.11',
                    'main-text.12'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='(Redirected from IBM Almaden Research Center)\nIBM Research is the research and development 
division for IBM, an American multinational information technology company headquartered in Armonk, New York, with 
operations in over 170 countries. IBM Research is the largest industrial research organization in the world and has
twelve labs on six continents. [1]',
            main_path='main-text.3',
            path_group=['main-text.2', 'main-text.3'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='(Redirected from IBM Almaden Research Center)\nIBM employees have garnered six Nobel Prizes, six
Turing Awards, 20 inductees into the U.S. National Inventors Hall of Fame, 19 National Medals of Technology, five 
National Medals of Science and three Kavli Prizes. $^{[2]}$ As of 2018, the company has generated more patents than
any other business in each of 25 consecutive years, which is a record. [3]',
            main_path='main-text.4',
            path_group=['main-text.2', 'main-text.4'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk="References\n1. 'Labs and locations ' (https://research.ibm.com/labs/). IBM Research. Archived 
(https://web.archiv e.org/web/20221221032216/https://research.ibm.com/labs/) from the original on 21 December",
            main_path='main-text.79',
            path_group=['main-text.78', 'main-text.79'],
            source_is_text=True
        )
    ]
)

In [9]:
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.retr_items[0])

The provenance of the answer can be inspected on the [source document](https://cps.foc-deepsearch.zurich.ibm.com/projects/e0ea87922f4b732407fb3b9cf3475f0edb90cc2d/library/private/d70f151acff22f19f9cfaffb1f5baa810c8de3db?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjJkNzBmMTUxYWNmZjIyZjE5ZjljZmFmZmIxZjViYWE4MTBjOGRlM2RiJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyYjMwYmM2NjdhMzI0YWUxMTFkMDI1NTI2NTYzYjY3NGE4ZDNmZDg2OWJjMDdjOGZkMjA0YWE5NWIwNWQ0MWYwYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMmIzMGJjNjY3YTMyNGFlMTExZDAyNTUyNjU2M2I2NzRhOGQzZmQ4NjliYzA3YzhmZDIwNGFhOTViMDVkNDFmMGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0EzJTdEJTdE).

### Semantic retrieval

Besides RAG, which includes natural language generation, a user may only be interested in
the semantic retrieval part.

This can be obtained very similarly to RAG, as shown below:

In [10]:
question = "Where was the first European IBM lab located?"

# submit natural-language query on document
question_query = SemanticQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional params
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    # rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = SearchResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk="Israel\nIBM Research-Haifa, previously known as the Haifa Research Lab (HRL) was founded as a 
small scientific center in 1972. $^{[55]}$ Since then, it has grown into a major lab that leads the development of 
innovative technologies and solutions for the IBM corporation. The lab's offices are situated in three locations 
across Israel: Haifa, Tel Aviv, and Beer Sheva.",
            main_path='main-text.62',
            path_group=['main-text.61', 'main-text.62'],
            source_is_text=True
        )
    ]
)

## RAG on document not in semantically indexed collection

### Prepare source

In [11]:
coords = ElasticProjectDataCollectionSource(
    proj_key=PROJ_KEY,
    index_key=SEM_OFF_IDX_KEY,
)
data_source = PrivateDataDocumentSource(
    source=coords,
    document_hash=SEM_OFF_IDX_DOC_HASH,
)

### Ingestion

In the cell below we show how to semantically index a single document:

In [12]:
# launch the ingestion of the document for DocumentQA
task = api.documents.semantic_ingest(
    project=PROJ_KEY,
    data_source=data_source,
    skip_ingested_docs=False,  # forcing re-indexing for the purpose of this example
)

# wait for the ingestion task to finish
api.tasks.wait_for(PROJ_KEY, task.task_id)

/Users/pva/work/github.com/DS4SD/deepsearch-examples/.venv/lib/python3.10/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `_LiteralGenericAlias` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'ing_out': {}}

Once the document has been semantically ingested, we can run both RAG and semantic retrieval queries against it, as shown below.

### RAG

In [13]:
question = "Which company created the first game console?"

# submit natural-language query on document
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=4,
    # text_weight=TEXT_WEIGHT,
    rerank=RERANK,

    ## optional generation params
    # model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q",
    # gen_params={"random_seed": 42, "max_new_tokens": 1024},
    # prompt_template="Answer the query based on the context.\n\nContext: {{ context }}\n\nQuery: {{ query }}",
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)


RAGResult(
    answers=[
        RAGAnswerItem(
            answer='Magnavox',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
                        chunk='Magnavox\nIn 1972 Magnavox introduced the Odyssey, the first video game console. 
$^{[10]}$ In 1975, the Magnavox Company was acquired by Philips of the Netherlands to ensure nationwide 
distribution for their VLP (later renamed LaserVision) Videodisc technology, and all Philips consumer electronics 
in the US under the Norelco name began rebranding them under the Magnavox name; $^{[11]}$ Philips acquired the 
similar-sounding company Philco in 1981, and Philips was able to freely use the Philips name, alternating with the 
Magnavox name for some electronics, with the personal care business continuing to use the Norelco name.',
                        main_path='main-text.9',
                        path_group=['main-text.5', 'main-text.9'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.3',
                    'main-text.4',
                    'main-text.5',
                    'main-text.6',
                    'main-text.7',
                    'main-text.9',
                    'main-text.10',
                    'main-text.11',
                    'main-text.12',
                    'main-text.13',
                    'main-text.14',
                    'main-text.15',
                    'main-text.16',
                    'main-text.17'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk='Magnavox\nIn 1972 Magnavox introduced the Odyssey, the first video game console. $^{[10]}$ In 
1975, the Magnavox Company was acquired by Philips of the Netherlands to ensure nationwide distribution for their 
VLP (later renamed LaserVision) Videodisc technology, and all Philips consumer electronics in the US under the 
Norelco name began rebranding them under the Magnavox name; $^{[11]}$ Philips acquired the similar-sounding company
Philco in 1981, and Philips was able to freely use the Philips name, alternating with the Magnavox name for some 
electronics, with the personal care business continuing to use the Norelco name.',
            main_path='main-text.9',
            path_group=['main-text.5', 'main-text.9'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk="Magnavox\nMagnavox (Latin for 'great voice ', stylized as MAGNAVOX) is an American electronics 
company that since 1975 has been a subsidiary of the Dutch electronics corporation Philips. [1] The predecessor to 
Magnavox was founded in 1911 by Edwin Pridham and Peter L. Jensen, coinventors of the moving-coil loudspeaker at 
their lab in Napa, California, under United States Patent number 1,105,924 for telephone receivers. $^{[2]}$ Six 
decades later, Magnavox produced the Odyssey, the world's first home video game console.",
            main_path='main-text.1',
            path_group=['main-text.0', 'main-text.1'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk='Magnavox\nDuring the late 1970s the company released the Odyssey², in Europe also known as 
Philips Videopac G7000.',
            main_path='main-text.12',
            path_group=['main-text.5', 'main-text.12'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
        

In [14]:
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.retr_items[0])

The provenance of the answer can be inspected on the [source document](https://cps.foc-deepsearch.zurich.ibm.com/projects/e0ea87922f4b732407fb3b9cf3475f0edb90cc2d/library/private/d0c6b811fa75fb8154bf7659162089a42fdd0895?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjJkMGM2YjgxMWZhNzVmYjgxNTRiZjc2NTkxNjIwODlhNDJmZGQwODk1JTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyMDI5MjEwZGY5MjljNzhlNzBkNzRlNmYxNDFhNDZkODMyNjkwNWNlNTg1NjJmMjA4MTgxOWM4MGMzOTIxZDVhMyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMjAyOTIxMGRmOTI5Yzc4ZTcwZDc0ZTZmMTQxYTQ2ZDgzMjY5MDVjZTU4NTYyZjIwODE4MTljODBjMzkyMWQ1YTMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0E5JTdEJTdE).

### Semantic retrieval

In [15]:
question = "Which company created the first game console?"

# submit natural-language query on document
question_query = SemanticQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional params
    retr_k=4,
    # text_weight=TEXT_WEIGHT,
    rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = SearchResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk='Magnavox\nIn 1972 Magnavox introduced the Odyssey, the first video game console. $^{[10]}$ In 
1975, the Magnavox Company was acquired by Philips of the Netherlands to ensure nationwide distribution for their 
VLP (later renamed LaserVision) Videodisc technology, and all Philips consumer electronics in the US under the 
Norelco name began rebranding them under the Magnavox name; $^{[11]}$ Philips acquired the similar-sounding company
Philco in 1981, and Philips was able to freely use the Philips name, alternating with the Magnavox name for some 
electronics, with the personal care business continuing to use the Norelco name.',
            main_path='main-text.9',
            path_group=['main-text.5', 'main-text.9'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk="Magnavox\nMagnavox (Latin for 'great voice ', stylized as MAGNAVOX) is an American electronics 
company that since 1975 has been a subsidiary of the Dutch electronics corporation Philips. [1] The predecessor to 
Magnavox was founded in 1911 by Edwin Pridham and Peter L. Jensen, coinventors of the moving-coil loudspeaker at 
their lab in Napa, California, under United States Patent number 1,105,924 for telephone receivers. $^{[2]}$ Six 
decades later, Magnavox produced the Odyssey, the world's first home video game console.",
            main_path='main-text.1',
            path_group=['main-text.0', 'main-text.1'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk='Magnavox\nDuring the late 1970s the company released the Odyssey², in Europe also known as 
Philips Videopac G7000.',
            main_path='main-text.12',
            path_group=['main-text.5', 'main-text.12'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            chunk="History\nJensen and Pridham founded the Commercial Wireless and Development Company in Napa, CA 
in 1911, moving to San Francisco, and then Oakland in 1916. In July 1917, a merger with The Sonora Phonograph 
Distributor Company was finalized and the Magnavox Company was born. Frank Morgan Steers was chosen as the 
company's first President. Jensen moved on to found the Jensen Radio Manufacturing Company in Chicago, in the late 
1920s. Pridham stayed on with Magnavox, which moved manufacturing to Fort Wayne, Indiana by the 1930s. The term 
'Commercial Wireless ' had a different meaning in the early days of radio and telephone. Magnavox manufactured 
radios, TVs, and phonographs. In the 1960s, Magnavox manufactured the first plasma displays for the military and 
for computer applications.",
            main_path='main-text.4',
            path_group=['main-text.3', 'main-text.4'],
            source_is_text=True
        )
    ]
)

---

## QA on document from public collection

### Prepare source

In [16]:
from deepsearch.cps.client.components.elastic import ElasticDataCollectionSource
from deepsearch.cps.client.components.documents import PublicDataDocumentSource

index_key = "acl"
document_hash = "0002e4fc1cef1c98b411c75e484db0a3d32f6bc1b4058e2985e5f377721761fb"

coords = ElasticDataCollectionSource(
    elastic_id="default",
    index_key=index_key,
)
data_source = PublicDataDocumentSource(
    source=coords,
    document_hash=document_hash,
)

### RAG

In [17]:
question = "How many goals can a player achieve in the MP game?"

# submit natural-language query on document
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=RETR_K,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

rich.print(rag_result)

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='A player can achieve 28 game goals in the MP game.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='0002e4fc1cef1c98b411c75e484db0a3d32f6bc1b4058e2985e5f377721761fb',
                        chunk="2.1 The MP Game and Dialogs\nIn total, the player can achieve up to 28 game goals by
conducting 12 separate dialogs in various parts of the virtual world. Each of the 12 dialogs in the MP game helps 
players to achieve the game goals. The player interacts with the virtual characters to obtain information that 
helps her to achieve these goals and, as a consequence, to increase her score in the game. Table 1 summarises the 
game goals and the contextual parameters (player's role, location in the virtual world, VCs present) associated 
with each dialog.",
                        main_path='main-text.24',
                        path_group=['main-text.22', 'main-text.24'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.16',
                    'main-text.17',
                    'main-text.19',
                    'main-text.20',
                    'main-text.21',
                    'main-text.22',
                    'main-text.23',
                    'main-text.24',
                    'main-text.25',
                    'main-text.26',
                    'main-text.28',
                    'main-text.29',
                    'main-text.30',
                    'main-text.31',
                    'main-text.32',
                    'main-text.33',
                    'main-text.34',
                    'main-text.35'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='0002e4fc1cef1c98b411c75e484db0a3d32f6bc1b4058e2985e5f377721761fb',
            chunk="2.1 The MP Game and Dialogs\nIn total, the player can achieve up to 28 game goals by conducting 
12 separate dialogs in various parts of the virtual world. Each of the 12 dialogs in the MP game helps players to 
achieve the game goals. The player interacts with the virtual characters to obtain information that helps her to 
achieve these goals and, as a consequence, to increase her score in the game. Table 1 summarises the game goals and
the contextual parameters (player's role, location in the virtual world, VCs present) associated with each 
dialog.",
            main_path='main-text.24',
            path_group=['main-text.22', 'main-text.24'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='0002e4fc1cef1c98b411c75e484db0a3d32f6bc1b4058e2985e5f377721761fb',
            chunk='2.1 The MP Game and Dialogs\nThe MP game is a multi-player quest where 3 teenagers seek to build
a joystick in order to free their uncle trapped in a video game $^{1}$. To build this joystick, the player (who 
alternatively represents anyone of these three teenagers) must explore the plastic factory and achieve 17 mandatory
goals (find the plans, get the appropriate mould, retrieve some plastic from the storing shed, etc), as well as 11 
optional goals which, when reached, provide them with extra information about the plastic industry (and therefore 
increases their knowledge of it).',
            main_path='main-text.23',
            path_group=['main-text.22', 'main-text.23'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='0002e4fc1cef1c98b411c75e484db0a3d32f6bc1b4058e2985e5f377721761fb',
            chunk='4.2 Preliminary User-Based Evaluation\nAfter playing, each user completed the questionnaire 
shown in Table 3. For those criteria such as dialog and game clarity, we do not report the scores since these are 
clearly impacted by how many times the player has played t